# KoBigBird KorNLI fine-tuning

긴 문장(최대 4096 토큰)이 입력 가능한 KoBigBird를 두 문장의 entail, contradiction, neutral을 판별할 수 있도록 fine - tuning 합니다.

# 데이터셋(KorNLI Datasets)

카카오브레인에서 공개한 문장 유사도 데이터셋 KorNLI를 사용합니다.


다운로드 링크:
https://github.com/kakaobrain/KorNLUDatasets/tree/master/KorNLI

KorSTS 데이터셋은 두 문장 쌍과 두 문장의 관계를 entailment, contradiction, neutral 세 개의 클래스 중 하나의 label이 부착된 데이터셋입니다.

BigBird에 두 문장 쌍을 입력하여 얻은 벡터 표현과 두 벡터의 차의 절대값을 concat하고 softmax를 거쳐 최종 분류를 하게 됩니다.

## Preprocessing

In [1]:
data_path = './data/KorNLI/'

### train, dev, test data 읽어오기

리스트의 각 원소는 리스트로 저장됩니다. [[문장1, 문장2], gold_label] 

In [2]:
import csv

train_mnli_data = []
train_snli_data = []
dev_data = []
test_data = []

with open(data_path + 'multinli.train.ko.tsv') as file:
    tsv_file = csv.reader(file, delimiter="\t", quoting=csv.QUOTE_NONE)
    for line in tsv_file:
        train_mnli_data.append([[line[0], line[1]], line[2]])
        
with open(data_path + 'snli_1.0_train.ko.tsv') as file:
    tsv_file = csv.reader(file, delimiter="\t", quoting=csv.QUOTE_NONE)
    for line in tsv_file:
        train_snli_data.append([[line[0], line[1]], line[2]])
        
with open(data_path + 'xnli.dev.ko.tsv') as file:
    tsv_file = csv.reader(file, delimiter="\t", quoting=csv.QUOTE_NONE)
    for line in tsv_file:
        dev_data.append([[line[0], line[1]], line[2]])
        
with open(data_path + 'xnli.test.ko.tsv') as file:
    tsv_file = csv.reader(file, delimiter="\t", quoting=csv.QUOTE_NONE)
    for line in tsv_file:
        test_data.append([[line[0], line[1]], line[2]])

In [3]:
train_mnli_data = train_mnli_data[1:]

In [4]:
train_snli_data = train_snli_data[1:]

In [5]:
dev_data = dev_data[1:]

In [6]:
test_data = test_data[1:]

In [7]:
train_data = train_mnli_data + train_snli_data

In [8]:
print('train_data 수:', len(train_data))
print('dev set 수:', len(dev_data))
print('test set 수:', len(test_data))

train_data 수: 942854
dev set 수: 2490
test set 수: 5010


### label Index 할당하기

In [9]:
label_to_idx = {'entailment':0, 'contradiction':1, 'neutral':2}
idx_to_label = {0:'entailment', 1:'contradiction', 2:'neutral'}

## 학습 준비

### GPU or CPU

In [10]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"{device}를 사용합니다.")

/home/user03/anaconda3/envs/koelectra/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda를 사용합니다.


### KoBigBird 모델, 토크나이저 불러오기

In [11]:
from transformers import  AutoModel,AutoTokenizer

model = AutoModel.from_pretrained("monologg/kobigbird-bert-base", attention_type="original_full", add_pooling_layer=False)
tokenizer = AutoTokenizer.from_pretrained("monologg/kobigbird-bert-base")

Some weights of the model checkpoint at monologg/kobigbird-bert-base were not used when initializing BigBirdModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'bert.pooler.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'bert.pooler.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BigBirdModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### 하이퍼파라미터

In [12]:
train_batch_size = 32
valid_bath_size = 16
epochs = 50
learning_rate=5e-5

## Dataset

데이터셋 클래스에 토크나이저를 받아와서 데이터셋 객체를 생성할 때 토크나이징 하여 리스트로 저장합니다.

In [13]:
from torch.utils.data import Dataset

class stsDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.sent1 = []
        self.sent2 = []
        self.label = []
        self.tokenizer = tokenizer
        for tup in data:
            self.sent1.append(self.tokenizer(tup[0][0], max_length=256, padding='max_length',return_tensors='pt'))
            self.sent2.append(self.tokenizer(tup[0][1], max_length=256, padding='max_length',return_tensors='pt'))
            self.label.append(label_to_idx[tup[1]])
        
    def __len__(self):
        return (len(self.label))
    
    def __getitem__(self, idx):
        return self.sent1[idx], self.sent2[idx], self.label[idx]

In [ ]:
train_dataset = stsDataset(train_data, tokenizer)
valid_dataset = stsDataset(test_data, tokenizer)

## DataLoader

모델 훈련 시에 지정한 배치 크기만큼 데이터를 반환해주는 DataLodader입니다.

DataLoader는 Dataset객체를 인자로 받습니다.

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    batch_size=train_batch_size
)

valid_dataloader = DataLoader(
    valid_dataset,
    shuffle=True,
    batch_size=valid_bath_size
)

## Train

### 훈련에 필요한 optimizer, 코사인 유사도 함수 손실함수를 정의합니다.

optimizer에는 모델의 파라미터를 넘겨주고 학습률을 지정해줘야 합니다.

다중 분류 모델이므로 CrossEntropyLoss를 사용합니다.

pytorch에서는 CrossEntropyLoss를 사용할 때 label을 one-hot 벡터로 변환해주지 않아도 됩니다. 또한 softmax가 포함되어 있기 때문에 그냥 모델의 logit을 전달하면 됩니다. label로는 위에서 딕셔너리로 정의한 정답 index를 전달하면 됩니다.

### 분류층을 추가한 custom 모델 클래스 정의

두 문장 각각 벡터와 두 벡터의 차이(element-wise difference)를 concat하여 클래스를 분류합니다. 
(sbert 논문 참고  https://arxiv.org/pdf/1908.10084.pdf)

In [ ]:
from torch import nn

class NLImodel(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.kobigbird = model
        self.classification_layer = nn.Linear(768 * 3, 3)
        
    def forward(self, b_sent1, b_sent2):
        sent1_vec = self.kobigbird(b_sent1['input_ids'].squeeze(1), b_sent1['attention_mask'].squeeze(1), b_sent1['token_type_ids'].squeeze(1))[0][:,0,:]
        sent2_vec = self.kobigbird(b_sent2['input_ids'].squeeze(1), b_sent2['attention_mask'].squeeze(1), b_sent2['token_type_ids'].squeeze(1))[0][:,0,:]
        
        difference = torch.sub(sent1_vec, sent2_vec)
        abs_difference = torch.abs(difference)
       
        concatenation = torch.cat((sent1_vec, sent2_vec, abs_difference), 1)
        pred = self.classification_layer(concatenation)
        
        return pred

In [ ]:
nli_model = NLImodel(model)
nli_model.to(device)

In [ ]:
from torch import nn

optimizer = torch.optim.Adam(nli_model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

### 훈련 및 검증 

최저 valid loss를 갱신할 때마다 모델을 저장합니다. 현재 디렉토리의 model 폴더에 저장됩니다.

In [ ]:
from tqdm import tqdm

In [ ]:
best_val_loss = 100

for epoch in range(1, epochs+1):
    
    model.train()
    
    train_loss = []
    step = 0
    
    for i, batch in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        
        batch = tuple(t.to(device) for t in batch)
        b_sent1, b_sent2, b_label = batch 
        
        logits = nli_model(b_sent1, b_sent2)
        
        loss = loss_fn(logits, b_label)
        loss.backward()
        
        optimizer.step()
        
        train_loss.append(loss)
        step += 1
        
    print(f"train {epoch} epoch 종료  평균 loss: {sum(train_loss)/step}")
    
    with torch.no_grad():
        
        model.eval()

        valid_loss = []
        step = 0

        for i, batch in enumerate(tqdm(valid_dataloader)):
            
            batch = tuple(t.to(device) for t in batch)
            b_sent1, b_sent2, b_label = batch # [batch_size, max_len, 768]

            logits = nli_model(b_sent1, b_sent2)
            
            loss = loss_fn(logits, b_label)

            valid_loss.append(loss)
            step += 1
            
            
        val_loss = sum(valid_loss)/step
        print(f"vaild {epoch} epoch 종료  평균 loss: {val_loss}")
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            model_save_path = './model/NLI/' +  'saved_model_epoch_' + str(epoch) + '.pt'
            model.save_pretrained(model_save_path)